In [1]:
import requests
import json
import dataset
import sqlite3

In [2]:
#Creating a class that will use a base url to get data fro shopify
class ShopifyScrapper():
    """
    base_url: It's the url of the website I intend to get dataset from
    page: A product page on Shopify can have a lot of pages. 
    parsejson: Adding '/products.json' to a shopify site will give you the json file of the page
    the parsejson method will grab data based on the arrangement of the json file. The method
    is suit for this particular project
    
    timeout: an inbuilt function that times out after the indicated seconds
    """
    def __init__(self,base_url):
        self.base_url = base_url
        
    def download_json(self, page):
        r = requests.get(self.base_url + f'/products.json?limit=250&page={page}',timeout=5)
        if r.status_code != 200:
            print('Bad Status Code: ', r.status_code)
        if len(r.json()['products']) > 0:
            data = r.json()['products']
            return data
        else:
            return
    def parsejson(self, jsondata):
        products = []
        for prod in jsondata:
            main_id = prod['id']
            title = prod['title']
            published_at = prod["published_at"]
            product_type = prod["product_type"]
            for variant in prod["variants"]:
                item = {
                    "id":main_id,
                    "title":title,
                    "published_at":published_at,
                    "product_type":product_type,
                    "varid":variant['id'],
                    "vartitle":variant['title'],
                    "sku":variant["sku"],
                    "price":variant["price"],
                    "available":variant["available"],
                    "created_at":variant["created_at"],
                    "updated_at":variant["updated_at"],
                    "compare_at_price":variant["compare_at_price"]
                }
                products.append(item)
        return products
    
            
        

In [3]:
def main():
    allbirds = ShopifyScrapper('https://www.allbirds.com/')
    result = []
    for page in range(1,10):
        data = allbirds.download_json(page)
        print(f'Getting page ----{page}')
        try:
            result.append(allbirds.parsejson(data))
        except:
            print(f'Completed. total_pages = {page-1}')
            break
    return result

In [4]:
products = main()
print(len(products))

Getting page ----1
Getting page ----2
Getting page ----3
Completed. total_pages = 2
2


In [5]:
total = [item for i in products for item in i]

In [6]:
import mysql.connector
from mysql.connector import Error

In [7]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [8]:
connection = create_server_connection("localhost", "root", "")

MySQL Database connection successful


In [9]:
connection.cursor()

In [10]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [11]:
query = "CREATE DATABASE Shopify;"
create_database(connection,query)

Database created successfully


In [12]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [13]:
create_db_connection("localhost", "root", "",'Shopify')

MySQL Database connection successful


In [14]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [15]:
create_product_table = """
CREATE TABLE products (
  id VARCHAR(200),
  title VARCHAR(200),
  published_at VARCHAR(200),
  product_type VARCHAR(200),
  varid VARCHAR(200) PRIMARY KEY,
  vartitle VARCHAR(200),
  sku VARCHAR(200),
  price VARCHAR(200),
  available VARCHAR(200),
  created_at VARCHAR(200),
  updated_at VARCHAR(200),
  compare_at_price VARCHAR(20)
  );
 """

connection = create_db_connection("localhost", "root","", "Shopify") # Connect to the Database
execute_query(connection, create_product_table) # Execute our defined query

MySQL Database connection successful
Query successful


In [16]:
for mydict in total:
    columns = ', '.join("`" + str(x).replace('/', '_') + "`" for x in mydict.keys())
    values = ', '.join("'" + str(x).replace('/', '_') + "'" for x in mydict.values())
    sql = "INSERT INTO %s ( %s ) VALUES ( %s );" % ('products', columns, values)
    #print(sql)
    connection = create_db_connection("localhost", "root","", "Shopify")
    execute_query(connection, sql)

MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection successful
Query successful
MySQL Database connection su

Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Breezers - Earth (Taupe Sole)', '2019-10-31T16:20:55-07:00', 'Shoes', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Breezers - Earth (Taupe Sole)', '2019-10-31T16:20:55-07:00', 'Shoes', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Breezers - Earth (Taupe Sole)', '2019-10-31T16:20:55-07:00', 'Shoes', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the 

MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool Runners - Pebble (White Sole)', '2019-10-31T16:20:55-07:00', 'Shoes', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool Runners - Sunrise (White Sole)', '2019-10-31T16:20:55-07:00', 'Shoes',...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool Runners - Sunrise (White Sole)', '2019-10-31T16:20:55-07:00', 'Shoes',...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to

MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Dasher Relay - Prisma Sesame (White Sole)', '2019-10-31T16:20:55-07:00...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Dasher Relay - Prisma Sesame (White Sole)', '2019-10-31T16:20:55-07:00...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Dasher Relay - Prisma Sesame (White Sole)', '2019-10-31T16:20:55-07:00...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to

Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's TrinoXO™ Tee - Classic Fit - Natural Black', '2019-10-31T16:20:55-07:00',...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's TrinoXO™ Tee - Classic Fit - Natural Black', '2019-10-31T16:20:55-07:00',...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's TrinoXO™ Tee - Classic Fit - Natural Black', '2019-10-31T16:20:55-07:00',...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right 

...

MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Loungers - Nikau Sapphire (White Sole)', '2019-10-31T16:20:55-07:00', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Loungers - Nikau Sapphire (White Sole)', '2019-10-31T16:20:55-07:00', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Loungers - Nikau Sapphire (White Sole)', '2019-10-31T16:20:55-07:00', ...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to

MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool & Tree Lounger_Tree Skipper Insoles - Natural Grey', '2019-10-31T16:20...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool & Tree Lounger_Tree Skipper Insoles - Natural Grey', '2019-10-31T16:20...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool & Tree Lounger_Tree Skipper Insoles - Natural Grey', '2019-10-31T16:20...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to

MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Loungers - Kauri Cobalt (White Sole)', '2019-10-31T16:20:55-07:00', 'S...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Loungers - Kauri Cobalt (White Sole)', '2019-10-31T16:20:55-07:00', 'S...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Loungers - Kauri Fiddle-Leaf (Fiddle-Leaf Sole)', '2019-10-31T16:20:55...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to

MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool & Tree Lounger_Tree Skipper Insoles - Natural Grey', '2019-10-31T16:20...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool & Tree Lounger_Tree Skipper Insoles - Natural Grey', '2019-10-31T16:20...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Wool & Tree Lounger_Tree Skipper Insoles - Natural Grey', '2019-10-31T16:20...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to

Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Runners - Kauri Marine Blue (White Sole)', '2019-10-31T16:20:55-07:00'...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Runners - Kauri Marine Blue (White Sole)', '2019-10-31T16:20:55-07:00'...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 's Tree Runners - Kauri Marine Blue (White Sole)', '2019-10-31T16:20:55-07:00'...' at line 1'
MySQL Database connection successful
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the 